“RESPUESTA A EJERCICIO 2 PASO V” Maquina serie N1 tipo standard-2 (2cpus, 7.5 Ram) 
    OS Debian se opta por este OS ya que es ligero y dado que se elige un tipo de maquina con especificaciones limitidas para ahorrar en costos me peracio la mejor 
    opcion 


Tomar el archivo "createvm.txt" para la creacion de la maquina virtual, se pude ejecutar en el bash de GCP, antes de ejecutar se debe crear o elegir un projecto y actualizar la etiqueta "-project" del codigo, se debe crear cueta de servicio y modicicar en la entiqueta "--service-accout"

Una vez creada la maquina virtual se debe registrar la instancia con la nueva API de Notebooks

Pasos: 1- En la consola de GCP dirigirse al servicio Vertex AI. 2- seleccionar Workbench. 3-Marcar la casilla Incluir instancias heredadas. 4- Click en REGISTER ALL

"RESPUESTA EJERCICIO 1 PASO II" Tomar el archivo "createbucket1.txt" para crea el bucket "anoc001-test-armandolara" en google cloud storage, se pude ejecutar en el bash de GCP en este paso los Buckes nacen con la propiedad de monitorear eventos atraves de Cloud Logging por lo que podemos crear procesos que se ejecuten atraves de eventos que sucedan dentro del bucket

"RESPUESTA EJERCICIO 1 PASO III" Tomar el archivo "createbucket2.txt" para crea el bucket "anoc001-test-data-armandolara" en google cloud storage, se pude ejecutar en el bash de GCP  en este paso los Buckes nacen con la propiedad de monitorear eventos atraves de Cloud Logging por lo que podemos crear procesos que se ejecuten atraves de eventos que sucedan dentro del bucket

Inicia desarrollo de proceso ETL, incluye respuestas a ejercicio 1 pasos: III,VI, VIII, IX, X

In [ ]:
pip install google-cloud-storage

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

In [1]:
import pandas as pd
from google.cloud import storage
import numpy as np

In [2]:
#Este bloque carga el archivo data_prueba_tecnica.csv al bucket anoc001-test-data-armandolara en la carpeta raw, si la carpeta no existe la crea
#Solo ejecutar este bloque si el entorno esta dentro de GCP

#variables de met
bucket_name = 'anoc001-test-data-armandolara'
bucket_file = 'raw/data_prueba_tecnica.csv' 
local_file = 'data_prueba_tecnica.csv'

#crea cliente-loggeo
storage_client = storage.Client()

#caraga archivo al bucket
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(bucket_file)
blob.upload_from_filename(local_file)

In [42]:
#Tomo el archivo que subimos del bucket y lo caraga en un dataframe
#En caso de correr esta el codigo en local cambiar el valor de la variable uri por path de archivo 


#uri = f'gs://{bucket_name}/{bucket_file}'
uri = local_file
df = pd.read_csv(uri, sep=',', encoding='UTF-8')

#Elimina las filas vacias RESPUESTA EJERCICIO 1 PASO VI A
df = df.dropna()


,id,name,company_id,amount,status,created_at,paid_at
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
9,cdf1a7a6f5c9b0db543371f9e4bd127b451c09c9,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,115.84,paid,2019-01-22,2019-01-22
11,90820e39fa6aa60555edb6855a6c83e59a9ba271,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.33,paid,2019-01-12,2019-01-12
12,315fb708b31d2b98da63db63481550cf01087c75,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,230.27,paid,2019-03-29,2019-03-29
...,...,...,...,...,...,...,...
9995,4907c36c037bfa4874047e14f722329016dc3908,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,ef9ec85887418d2ccf220686312cfc4c615665d5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,50bb3950ecde94ca3e07853003a11a413508da36,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19
9998,1b2c4e503a4a952064b70369cf8e72937c720474,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.42,paid,2019-03-10,2019-03-10


In [43]:
#En este etapa identificamos combinaciones unicas de los campos name y company_id antes de crear df agregado
cat = pd.DataFrame() 
cat_comp = pd.DataFrame() 

for f in df[['name']]['name'].unique():
    df_cat = pd.DataFrame(columns=['company_id','name']) 
    df_cat['company_id'] = pd.DataFrame(df[df['name']== f ]['company_id'].unique())
    df_cat['name'] = f
    cat = pd.concat([cat,df_cat],axis = 0, ignore_index=True)

for f in df[pd.isnull(df['name'])]['company_id'].unique():
    df_cat_comp = pd.DataFrame(columns=['company_id','name']) 
    df_cat_comp['company_id'] = pd.DataFrame(df[df['company_id']== f ]['company_id'].unique())
    #df_cat_comp['name'] = None
    cat = pd.concat([cat,df_cat_comp],axis = 0, ignore_index=True)
    
cat.sort_values(['company_id'])

,company_id,name
2,*******,MiPasajefy
3,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
4,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPas0xFFFF
5,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiP0xFFFF
6,cbf1c8b09cd5b549416d49d220a40cbd317f952e,NaN
1,NaN,MiPasajefy


RESPUESTA EJERCICIO 1 PASO VI B Identifique que existen valores en los que un mismo company_id tiene 3 diferentes etiquetas name y existen voleres nulos  y un valor ******* en company_id

In [44]:
#creo un catalogo con valores unicos y limpios
name = ['MiPas0xFFFF','MiP0xFFFF']
cleancat = cat[~cat['name'].isin(name) & cat.name.notnull() & cat.company_id.notnull()].copy()
cleancat = cleancat[cleancat['company_id'] != '*******']
cleancat.reset_index()


,index,company_id,name
0,0,cbf1c8b09cd5b549416d49d220a40cbd317f952e,MiPasajefy
1,3,8f642dc67fccf861548dfe1c761ce22f795e91f0,Muebles chidos


In [45]:
#cruzo el df raw vs el catalogo de atrimutos para limpiar los campos name y company 
#limpiamos el campo name
join = pd.merge(df, cleancat, on=["company_id"], how="left")
join['name'] = np.where(join['name_y'].isnull(),join['name_x'],join['name_y'])

In [46]:
#limpiamos el camco companany_id
join = pd.merge(join, cleancat, on=["name"], how="left")
join['company_id'] = np.where(join['company_id_y'].isnull(),join['company_id_x'],join['company_id_y'])

In [47]:
#creamos un df dode dejamos la informacion procesada
df_procesed = join[['id','name','company_id','amount','status','created_at','paid_at']]
df_procesed

,id,name,company_id,amount,status,created_at,paid_at
0,48ba4bdbfb56ceebb32f2bd0263e759be942af3d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-03-19,NaN
1,05fc6f5ac66b6ee7e4253aa5d0c2299eb47aaaf4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,pending_payment,2019-05-06,NaN
2,2cdce231c1fc6a2061bfa2f1d978351fe217245d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,3.00,voided,2019-02-22,NaN
3,81633ba310a50b673efd469c37139576982901aa,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,102.61,paid,2019-02-27,2019-02-27
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
...,...,...,...,...,...,...,...
9995,4907c36c037bfa4874047e14f722329016dc3908,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,59.88,paid,2019-03-14,2019-03-14
9996,ef9ec85887418d2ccf220686312cfc4c615665d5,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,23.05,paid,2019-02-20,2019-02-20
9997,50bb3950ecde94ca3e07853003a11a413508da36,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,20.50,paid,2019-03-19,2019-03-19
9998,1b2c4e503a4a952064b70369cf8e72937c720474,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,81.42,paid,2019-03-10,2019-03-10


In [50]:
pd.DataFrame(df_procesed['company_id'].unique())

,0
0,cbf1c8b09cd5b549416d49d220a40cbd317f952e
1,8f642dc67fccf861548dfe1c761ce22f795e91f0


In [49]:
def loadstorace(bucket, dfpast, name):
    bucket = storage_client.bucket(bucket)
    df_procesed = dfpast
    bucket.blob(name).upload_from_string(df_procesed.to_csv(), 'text/csv') 

In [40]:
#Tomo el dataframe processed y lo cargo al bucket anoc001-test-data-armandolara, anoc001-test-armandolara  de storage en la campeta processed
#Solo ejecutar este bloque si el entorno esta dentro de GCP

bucketname = ['anoc001-test-data-armandolara' , 'anoc001-test-armandolara']
for i in bucketname:
    loadstorace (i, df_procesed,'processed/data_prueba_tecnica_procesed.csv')


In [22]:
pip install -U pandasql


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [30]:
#EN ESTE BLOQUE SE CREA DATAFRAME INSUMO PARA EL DASHBOARD CON LOS DATOS AGREGADOS
from pandasql import sqldf

q = """
SELECT 
    name
    ,company_id
    ,created_at
    , sum(amount) AS Total
FROM df_procesed 
GROUP BY     
    name
    ,company_id
    ,created_at
"""
dfgroupby = sqldf(q, globals())

In [31]:
dfgroupby

,name,company_id,created_at,Total
0,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2019-01-01,4150.04
1,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2019-01-02,17044.92
2,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2019-01-03,6735.66
3,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2019-01-04,6349.69
4,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,2019-01-05,5184.97
...,...,...,...,...
204,Muebles chidos,8f642dc67fccf861548dfe1c761ce22f795e91f0,2019-05-08,519.00
205,Muebles chidos,8f642dc67fccf861548dfe1c761ce22f795e91f0,2019-05-10,6248.00
206,Muebles chidos,8f642dc67fccf861548dfe1c761ce22f795e91f0,2019-05-12,2199.00
207,Muebles chidos,8f642dc67fccf861548dfe1c761ce22f795e91f0,2019-05-14,8099.10


In [27]:
#GUARDA EL DF EN CSV PARA CONSUMO
#Insumo para dashbord
dfgroupby.to_csv('group.csv')

In [41]:
#Tomo el dataframe dfgroupby y lo cargo al bucket anoc001-test-data-armandolara, anoc001-test-armandolara  de storage en la campeta processed
#Insumo para dashbord
#Solo ejecutar este bloque si el entorno esta dentro de GCP

bucketname = ['anoc001-test-data-armandolara' , 'anoc001-test-armandolara']
for i in bucketname:
    loadstorace (i, dfgroupby,'processed/dash_insumo.csv')

TODO EL CODIGO DE ESTE NOTBOOK SE DEJA LISTO EN LA CARPETA "FUNCTIONS" PARA QUE SEA DESPLEGADO COMO FUNCION DENTRO DE GCP CLOUD FUNCTIONS Y SEA EJECUTADO ATRAVES DE EVENTOS "AL CARGAR NUEVOS ARCHIVOS EN LA CARPETA RAW DE ALGUNO DE LOS BUCKETS" 